In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-appetency/sample_submission.csv
/kaggle/input/netflix-appetency/train.csv
/kaggle/input/netflix-appetency/test.csv


In [2]:
import pandas as pd

In [3]:
%%capture
pip install autogluon

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [5]:
train_data = TabularDataset("../input/netflix-appetency/train.csv")

train_data.head()

,id,target,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_497,feature_498,feature_499,feature_500,feature_501,feature_502,feature_503,feature_504,feature_505,feature_506
0,0,0,C0,C0,C1,C5,C11,37.56,54.756667,54.756667,...,0,0,0,0,0,0,0,0,0,0
1,1,0,C0,C0,C3,C5,C1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,0,C0,C0,C3,C5,C2,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,0,C0,C0,C1,C5,C1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,1,C0,C0,C3,C3,C11,37.48,37.480000,37.161333,...,0,0,0,0,0,0,0,0,0,0


In [6]:
label = 'target'
print("Summary of target variable: \n", train_data[label].describe())

Summary of target variable: 
 count    70000.000000
mean         0.298186
std          0.457465
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: target, dtype: float64


In [7]:
train=pd.read_csv("../input/netflix-appetency/train.csv")
test=pd.read_csv("../input/netflix-appetency/test.csv")
sample=pd.read_csv("../input/netflix-appetency/sample_submission.csv")

In [8]:
train.shape

(70000, 509)

In [9]:
train.head()

,id,target,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_497,feature_498,feature_499,feature_500,feature_501,feature_502,feature_503,feature_504,feature_505,feature_506
0,0,0,C0,C0,C1,C5,C11,37.56,54.756667,54.756667,...,0,0,0,0,0,0,0,0,0,0
1,1,0,C0,C0,C3,C5,C1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,0,C0,C0,C3,C5,C2,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,0,C0,C0,C1,C5,C1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,1,C0,C0,C3,C3,C11,37.48,37.480000,37.161333,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# time_limit = 60  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
# metric = 'roc_auc'  # specify your evaluation metric here
# predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')
# predictor.leaderboard(test_data, silent=True)

In [11]:
# save_path = './agModels-predictClass'  # specifies folder to store trained models
# predictor = TabularPredictor(label=label, path=save_path).fit(train_data,eval_metric)

In [12]:
#save_path = './agModels-predictTarget'
time_limit = 3600*9  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
import autogluon.core as ag

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 50,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 10000,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

#time_limit = 2*60  # train various models for ~2 min
num_trials = 50  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train, time_limit=time_limit,holdout_frac=0.6,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

# predictor = TabularPredictor(label,path=save_path, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')
# #predictor.leaderboard(test_data, silent=True)

  0%|          | 0/50 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a f

  0%|          | 0/50 [00:00<?, ?it/s]

#0.780519

In [13]:
predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.778710     198.866238  10036.301281                0.018389          14.966882            2       True         57
1     NeuralNetMXNet/T4   0.769092      38.235268   2166.468697               38.235268        2166.468697            1       True         55
2          LightGBM/T20   0.768225       0.503356     13.153476                0.503356          13.153476            1       True         21
3          LightGBM/T32   0.768136       0.604022     12.298735                0.604022          12.298735            1       True         33
4          LightGBM/T42   0.768066       1.712092     36.346077                1.712092          36.346077            1       True         43
5          LightGBM/T34   0.767481       1.000243     17.497912                1.00024

{'model_types': {'LightGBM/T0': 'LGBModel',
  'LightGBM/T1': 'LGBModel',
  'LightGBM/T2': 'LGBModel',
  'LightGBM/T3': 'LGBModel',
  'LightGBM/T4': 'LGBModel',
  'LightGBM/T5': 'LGBModel',
  'LightGBM/T6': 'LGBModel',
  'LightGBM/T7': 'LGBModel',
  'LightGBM/T8': 'LGBModel',
  'LightGBM/T9': 'LGBModel',
  'LightGBM/T10': 'LGBModel',
  'LightGBM/T11': 'LGBModel',
  'LightGBM/T12': 'LGBModel',
  'LightGBM/T13': 'LGBModel',
  'LightGBM/T14': 'LGBModel',
  'LightGBM/T15': 'LGBModel',
  'LightGBM/T16': 'LGBModel',
  'LightGBM/T17': 'LGBModel',
  'LightGBM/T18': 'LGBModel',
  'LightGBM/T19': 'LGBModel',
  'LightGBM/T20': 'LGBModel',
  'LightGBM/T21': 'LGBModel',
  'LightGBM/T22': 'LGBModel',
  'LightGBM/T23': 'LGBModel',
  'LightGBM/T24': 'LGBModel',
  'LightGBM/T25': 'LGBModel',
  'LightGBM/T26': 'LGBModel',
  'LightGBM/T27': 'LGBModel',
  'LightGBM/T28': 'LGBModel',
  'LightGBM/T29': 'LGBModel',
  'LightGBM/T30': 'LGBModel',
  'LightGBM/T31': 'LGBModel',
  'LightGBM/T32': 'LGBModel',
  'Li

In [14]:
test_data = TabularDataset("../input/netflix-appetency/test.csv")
#y_test = test_data[label]  # values to predict
test_data_nolab = test_data.copy()  # delete label column to prove we're not cheating
test_data_nolab.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_497,feature_498,feature_499,feature_500,feature_501,feature_502,feature_503,feature_504,feature_505,feature_506
0,5,C0,C0,C0,C5,C2,37.638,36.361667,42.688333,31.66,...,0,0,0,0,0,0,0,0,0,0
1,7,C0,C0,C3,C5,C11,32.540,32.540000,39.403333,30.82,...,0,0,0,0,0,0,0,0,0,0
2,9,C0,C0,C1,C5,C1,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,11,C0,C0,C3,C5,C2,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,14,C0,C0,C1,C5,C10,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
#perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Predictions:  
 0        0
1        0
2        0
3        0
4        0
        ..
29995    1
29996    0
29997    0
29998    1
29999    0
Name: target, Length: 30000, dtype: int64


In [16]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.778710     198.866238  10036.301281                0.018389          14.966882            2       True         57
1     NeuralNetMXNet/T4   0.769092      38.235268   2166.468697               38.235268        2166.468697            1       True         55
2          LightGBM/T20   0.768225       0.503356     13.153476                0.503356          13.153476            1       True         21
3          LightGBM/T32   0.768136       0.604022     12.298735                0.604022          12.298735            1       True         33
4          LightGBM/T42   0.768066       1.712092     36.346077                1.712092          36.346077            1       True         43
5          LightGBM/T34   0.767481       1.000243     17.497912                1.00024

In [17]:
# pred_probs=predictor.predict_proba(test_data_nolab, model='LightGBMXT_BAG_L1')
# pred_probs

In [18]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

,0,1
0,0.610258,0.389742
1,0.726641,0.273359
2,0.774019,0.225981
3,0.892508,0.107492
4,0.808993,0.191007


In [19]:
data=pd.concat([test['id'],pred_probs[1]],axis=1, ignore_index=True)
data

,0,1
0,5,0.389742
1,7,0.273359
2,9,0.225981
3,11,0.107492
4,14,0.191007
...,...,...
29995,99981,0.566372
29996,99982,0.272700
29997,99992,0.140996
29998,99993,0.511521


In [20]:
data.columns=['id','target']

In [21]:
data.set_index('id').to_csv("Netflix4.csv")